# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II.

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Helper Functions: read_line and run_query

In [8]:
def read_line(line):
    """
    Decode the lines read from the csv file and store them in a dictionary to make it easier to manipulate.
    
    Args:
        line (:obj:`list`): One object of class list representing one row of csv file.
    
    Returns:
        record (:obj:`dict`): Dictionary with the row decoded in different named fields.
    """
    
    record = {
        "artist" : line[0],
        "firstname" : line[1],
        "gender" : line[2],
        "iteminsession" : int(line[3]),
        "lastname" : line[4],
        "length" : float(line[5]),
        "level" : line[6],
        "location" : line[7],
        "sessionid" : int(line[8]),
        "song" : line[9],
        "userid" : int(line[10])
    }
    return record

In [9]:
def run_query(session, query):
    """
    Execute the query passed as argument in Apache Cassandra using the session to the database.
    
    Args:
        session (:obj:`session`): One object of class session to the Apache Cassandra database.
        query (str): Query to be executed in Apache Cassandra
    
    Returns:
        df (:obj:`DataFrame`): Pandas DataFrame representing the results of the query in Apache Cassandra.
    """
    try:
        rows = session.execute(query)
        df = pd.DataFrame(list(rows))
    except Exception as e:
        print(e)
    
    return df

#### Table 1: session_songs
_Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4_

In [10]:
''' 
    Table created to provide an fast query on songs played by users on a specific track of one session. 
    It is specifically designed to supply the requirements of a query like stated in item 1:
        -> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
    The INSERT and SELECT are also designed to deliver the analytics team request on that query.
'''

query = "CREATE TABLE IF NOT EXISTS session_songs "
query = query + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        row = read_line(line)
        session.execute(query, (row["sessionid"], row["iteminsession"], row["artist"], row["song"], row["length"]))

#### Description of the model
In the session_songs table the Primary Key has two fields: session id and item session id. sessionid is used as the partition key and iteminsession as clustering keys. Partitioning is done by session and within that partition, rows are ordered by the iteminsession. Since we need to know a specific play occurance of one song, we created the table with this level of detail. Session Id along with the Item Session Id seems to be the most detailed information available in event datafile and that was what we decided to keep in session_songs in order to address the requirement of this query.

In [12]:
query = "select sessionid, iteminsession, artist, song, length from session_songs WHERE sessionid = 338 and iteminsession = 4"
run_query(session, query)

,sessionid,iteminsession,artist,song,length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Table 2: users_plays
_Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182_

In [13]:
''' 
    Table created to provide an fast query on songs played by users on a specific session. 
    It is specifically designed to supply the requirements of a query like stated in item 2:
        -> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    The INSERT and SELECT are also designed to deliver the analytics team request on that query.
'''

query = "CREATE TABLE IF NOT EXISTS users_plays "
query = query + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, PRIMARY KEY (userid, sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)             

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_plays (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        row = read_line(line)
        session.execute(query, (row["userid"], row["sessionid"], row["iteminsession"], row["artist"], row["song"], row["firstname"], row["lastname"]))

#### Description of the model
In the users_plays table the Primary Key has three fields: user id, session id and item session id. User id is used as the partition key while session id and iteminsession as clustering keys. Partitioning is done by user id and within that partition, rows are ordered by the session id and iteminsession. It is important for this query to obtain the songs ordered by item session id and that made us to model the table with session id and item session id as clustering keys. 

In [15]:
query = "select artist, iteminsession, song, firstname, lastname from users_plays WHERE userid = 10 and sessionid = 182"
run_query(session, query)

,artist,iteminsession,song,firstname,lastname
0,Down To The Bone,0,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,1,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,2,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,3,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Table 3: song_fans
_Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'_

In [16]:
''' 
    Table created to provide an fast query on users that played a specific song. 
    It is specifically designed to supply the requirements of a query like stated in item 3:
        -> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
    The INSERT and SELECT are also designed to deliver the analytics team request on that query.
'''

query = "CREATE TABLE IF NOT EXISTS song_fans "
query = query + "(song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_fans (song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        row = read_line(line)
        session.execute(query, (row["song"], row["userid"], row["firstname"], row["lastname"]))

#### Description of the model
In the song_fans table the Primary Key has two fields: song name and user id. Song name is used as the partition key and user id as clustering keys. Partitioning is done by song name and within that partition, rows are ordered by the user id. Since we only want to provide the users that have listened to specific songs, we did not add more fields from events. In particular, we are not interested to know how many times the song was played in total nor how many times a specific user played that song. Song name and user id suffice our requirements here as they together identify uniquely what is important for the query.

In [18]:
query = "select firstname, lastname from song_fans WHERE song = 'All Hands Against His Own'"
run_query(session, query)

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [19]:
query = "drop table session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table users_plays"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_fans"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()